In [ ]:
pip install titlecase

In [25]:
import pandas as pd
import titlecase
from unidecode import unidecode

# Ler o arquivo CSV
df = pd.read_csv('artigos_scopuswos_hindexadicionado.csv')

# Definir uma função para remover os parênteses dos nomes
def remove_parentheses(full_names):
    if pd.notna(full_names):
        names = full_names.split('; ')
        cleaned_names = [name.split(' (')[0] for name in names]
        return '; '.join(cleaned_names)
    else:
        return full_names

# Aplicar a função à coluna 'author_full_names' e corrigir nomes desconfigurados
df['author_full_names'] = df['author_full_names'].apply(remove_parentheses)
df['author_full_names'] = df['author_full_names'].apply(lambda x: unidecode(x) if pd.notna(x) else x)

# Definir uma função para capitalizar apenas a primeira letra da primeira palavra do título usando titlecase
def capitalize_first_word(title):
    if pd.notna(title):
        return titlecase.titlecase(title)
    else:
        return title

# Aplicar a função à coluna 'title'
df['title'] = df['title'].apply(capitalize_first_word)

# Salvar o DataFrame de volta no arquivo CSV
df.to_csv('artigos_scopuswos_hindexadicionado2.csv', index=False)


In [26]:
import pandas as pd

# Ler o arquivo CSV
csv_file = 'artigos_scopuswos_hindexadicionado2.csv'
df = pd.read_csv(csv_file)

# Nome do arquivo de saída BibTeX
bibtex_file = 'output.bib'

# Função para criar uma entrada BibTeX para cada linha no DataFrame
def create_bibtex_entry(row):
    author_full_names = row['author_full_names']
    title = row['title']
    publisher = row['publisher']
    year = row['year']
    journal = row['journal_title']
    volume = row['volume']
    issue = row['issue']
    page_start = row['page_start']
    page_end = row['page_end']
    url = row['link']
    abstract = row['abstract']
    doi = row['doi']

    # Verificar valores nulos e substituí-los por strings vazias
    author_full_names = author_full_names if not pd.isna(author_full_names) else ""
    title = title if not pd.isna(title) else ""
    publisher = publisher if not pd.isna(publisher) else ""
    year = int(year) if not pd.isna(year) else ""
    journal = journal if not pd.isna(journal) else ""
    volume = str(volume) if not pd.isna(volume) else ""
    issue = str(issue) if not pd.isna(issue) else ""
    page_start = str(page_start) if not pd.isna(page_start) else ""
    page_end = str(page_end) if not pd.isna(page_end) else ""
    url = url if not pd.isna(url) else ""
    abstract = abstract if not pd.isna(abstract) else ""
    doi = doi if not pd.isna(doi) else ""

    bibtex_entry = f"@article{{\n\
    author = {{{author_full_names}}},\n\
    title = {{{title}}},\n\
    publisher = {{{publisher}}},\n\
    year = {{{year}}},\n\
    journal = {{{journal}}},\n\
    volume = {{{volume}}},\n\
    number = {{{issue}}},\n\
    pages = {{{page_start}--{page_end}}},\n\
    url = {{{url}}},\n\
    abstract = {{{abstract}}},\n\
    doi = {{{doi}}}\n\
}}\n"

    return bibtex_entry

# Aplicar a função de criação de entradas BibTeX a cada linha
df['bibtex_entry'] = df.apply(create_bibtex_entry, axis=1)

# Filtrar entradas vazias
df = df[df['bibtex_entry'] != ""]

# Salvar as entradas BibTeX em um arquivo
with open(bibtex_file, 'w', encoding='utf-8') as f:
    for entry in df['bibtex_entry']:
        f.write(entry)
        f.write('\n')

print(f'Arquivo BibTeX gerado com sucesso: {bibtex_file}')


Arquivo BibTeX gerado com sucesso: output.bib
